## Deep Research with Google Gemini

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research workflow is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [ ]:
from google import genai
from google.genai import types
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict, List
from IPython.display import display, Markdown
import json

In [ ]:
load_dotenv(override=True)

## Using Google Search for Grounding

The Gemini API can use Google Search as a tool to ground its responses in up-to-date, real-world information. This is a powerful feature for research tasks.

In [ ]:
client = genai.Client()

In [ ]:
async def search_the_web(query: str):
    """Performs a grounded search using the Gemini API and Google Search."""
    print(f"Executing search for: '{query}'")
    
    instructions = "You are a research assistant. Given a search term, you search the web for that term and produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 words. Capture the main points. Write succinctly. This will be consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. Do not include any additional commentary other than the summary itself."
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[query],
        config=types.GenerateContentConfig(
            system_instruction=instructions,
            tools=[types.Tool(google_search=types.GoogleSearch())]
        ),
    )
    
    summary = response.text
    search_queries = response.candidates[0].grounding_metadata.web_search_queries
    
    print(f"\t> Grounded with search queries: {search_queries}")
    
    return summary

# Example usage
result_summary = await search_the_web("Latest AI Agent frameworks in 2025")
display(Markdown(result_summary))

### We will now use Structured Outputs to create a research plan.

In [ ]:
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: List[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

In [ ]:
async def plan_searches(query: str, num_searches: int = 3) -> WebSearchPlan:
    """Uses Gemini to create a research plan with structured output."""
    print("Planning searches...")
    
    instructions = f"You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output {num_searches} terms to query for."
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[query],
        config=types.GenerateContentConfig(
            system_instruction=instructions,
            response_mime_type='application/json',
            response_schema=WebSearchPlan,
        )
    )
    
    plan = WebSearchPlan.model_validate_json(response.text)
    print(f"Will perform {len(plan.searches)} searches.")
    return plan

# Example usage
search_plan = await plan_searches("Latest AI Agent frameworks in 2025")
print(json.dumps(search_plan.model_dump(), indent=2))

In [ ]:
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    try:
        sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
        from_email = Email("test@example.com") # Change this to your verified email
        to_email = To("test@example.com") # Change this to your email
        content = Content("text/html", html_body)
        mail = Mail(from_email, to_email, subject, content).get()
        sg.client.mail.send.post(request_body=mail)
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

In [ ]:
class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: List[str] = Field(description="Suggested topics to research further")

### Orchestration: Putting it all together

In [ ]:
async def perform_searches(search_plan: WebSearchPlan) -> List[str]:
    """ Call search_the_web() for each item in the search plan """
    print("\nPerforming searches...")
    tasks = [asyncio.create_task(search_the_web(item.query)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching.")
    return results

async def write_report(query: str, search_results: List[str]) -> ReportData:
    """ Use Gemini to write a report based on the search results"""
    print("\nWriting report...")
    
    instructions = (
        "You are a senior researcher tasked with writing a cohesive report for a research query. "
        "You will be provided with the original query, and some initial research done by a research assistant.\n"
        "You should first come up with an outline for the report that describes the structure and "
        "flow of the report. Then, generate the report and return that as your final output.\n"
        "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
        "for 5-10 pages of content, at least 1000 words."
    )
    
    input_prompt = f"Original query: {query}\n\nSummarized search results:\n{'\n---\n'.join(search_results)}"
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[input_prompt],
        config=types.GenerateContentConfig(
            system_instruction=instructions,
            response_mime_type='application/json',
            response_schema=ReportData,
        )
    )
    
    report = ReportData.model_validate_json(response.text)
    print("Finished writing report.")
    return report

async def email_report(report: ReportData):
    """ Formats and sends the final report via email."""
    print("\nFormatting and sending email...")
    subject = f"Research Report: {report.short_summary}"
    send_email(subject, report.markdown_report)
    print("Email sent.")
    return report

### Showtime!

In [ ]:
async def main():
    query = "Latest AI Agent frameworks in 2025"

    print("Starting research...")
    # 1. Plan the research
    search_plan = await plan_searches(query)
    
    # 2. Execute the searches
    search_results = await perform_searches(search_plan)
    
    # 3. Write the final report
    report = await write_report(query, search_results)
    
    # 4. Email the report
    await email_report(report)  
    
    print("\nHooray! Research complete.")
    display(Markdown(f"## Report Summary\n{report.short_summary}"))
    display(Markdown(f"### Follow-up Questions\n* {'\n* '.join(report.follow_up_questions)}"))
    
await main()

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress!</h2>
            <span style="color:#00cc00;">You've reached an important moment; you've created a valuable research workflow using the native Gemini API. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!
            </span>
        </td>
    </tr>
</table>